In [1]:
!git clone https://github.com/jan1na/Neural-Cellular-Automata.git

%cd Neural-Cellular-Automata

Cloning into 'Neural-Cellular-Automata'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 90 (delta 54), reused 42 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 275.06 KiB | 11.96 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/Neural-Cellular-Automata


In [2]:
!pip install medmnist torch torchvision tqdm

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from medmnist import PathMNIST
from train_utils import train, evaluate, plot_training_history
from models import NCA
import os

# Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256

In [3]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = PathMNIST(split='train', transform=transform, download=True)
val_dataset = PathMNIST(split='val', transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

100%|██████████| 206M/206M [03:13<00:00, 1.06MB/s]


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NCA().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
NUM_EPOCHS = 50

best_acc = 0
model_path = '/content/drive/MyDrive/NCA/best_nca_pathmnist.pth'
os.makedirs(os.path.dirname(model_path), exist_ok=True)

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device, is_NCA=True)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device, is_NCA=True)
    print(f"Epoch {epoch:02d}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f"New best model saved to {model_path}")

    if val_acc > 0.90:
        print("Converged (validation accuracy > 90%)")
        break

print("Training completed.")
plot_training_history(train_losses, train_accuracies, val_losses, val_accuracies, plot_save_path)

Epoch 01: Train Acc: 0.5730, Val Acc: 0.6137
New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth


Epoch 02: Train Acc: 0.7445, Val Acc: 0.7862
New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth


Epoch 03: Train Acc: 0.8277, Val Acc: 0.8214
New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth


Epoch 04: Train Acc: 0.8673, Val Acc: 0.8761
New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth


Epoch 05: Train Acc: 0.8872, Val Acc: 0.8868
New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth


Epoch 06: Train Acc: 0.9005, Val Acc: 0.9204
New best model saved to /content/drive/MyDrive/NCA/best_nca_pathmnist.pth
Converged (validation accuracy > 90%)
Training completed.
